In [1]:
!pip install scikit-learn



In [2]:
!pip install --upgrade pixiedust

In [3]:
pip install --upgrade jinja2==3.0.2

  Using cached Jinja2-3.0.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached Jinja2-3.0.2-py3-none-any.whl (133 kB)
Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install brunel

!pip install ibm_watson_machine_learning

import pixiedust

import sklearn

import pandas as pd

from sklearn.ensemble import RandomForestClassifier 

import numpy as np 

from sklearn.model_selection import train_test_split 

from scipy.io import arff 

import brunel

from ibm_watson_machine_learning import APIClient

Pixiedust database opened successfully


Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [2]:
raw_df=pixiedust.sampleData('https://raw.githubusercontent.com/PMNJAYANTHSAI/ibm-data/main/AutoInsClaims.csv')


Downloaded 274837 bytes
Creating pandas DataFrame for 'https://raw.githubusercontent.com/PMNJAYANTHSAI/ibm-data/main/AutoInsClaims.csv'. Please wait...
Loading file using 'pandas'
Successfully created pandas DataFrame for 'https://raw.githubusercontent.com/PMNJAYANTHSAI/ibm-data/main/AutoInsClaims.csv'


In [6]:
pip install brunel


Note: you may need to restart the kernel to use updated packages.


In [ ]:
display(raw_df)

In [7]:
raw_df["EXPIRY_DATE"] = pd.to_datetime(raw_df["EXPIRY_DATE"])

raw_df["LOSS_EVENT_TIME"] = pd.to_datetime(raw_df["LOSS_EVENT_TIME"])

 

raw_df["DAYS_FROM_LOSS"] = raw_df["LOSS_EVENT_TIME"] - raw_df["EXPIRY_DATE"]

raw_df["DAYS_FROM_LOSS"] = abs(raw_df.DAYS_FROM_LOSS.dt.days)

 

raw_df.loc[raw_df['DAYS_FROM_LOSS'] >= 15, 'SUSPICIOUS_CLAIM_TIME'] = 1

raw_df.loc[raw_df['DAYS_FROM_LOSS'] < 15, 'SUSPICIOUS_CLAIM_TIME'] = 0

In [8]:
raw_df["SUSPICIOUS_CLAIM_TIME"].value_counts()

1.0    915
0.0     60
Name: SUSPICIOUS_CLAIM_TIME, dtype: int64

In [9]:
# Expired drivers license (if date of loss > insurance_driver.drivers_license_expiry)

raw_df["DRIVERS_LICENSE_EXPIRY"] = pd.to_datetime(raw_df["DRIVERS_LICENSE_EXPIRY"])

 

raw_df["DAYS_FROM_L_EXPIRY"] = raw_df["DRIVERS_LICENSE_EXPIRY"] - raw_df["LOSS_EVENT_TIME"]

raw_df["DAYS_FROM_L_EXPIRY"] = raw_df.DAYS_FROM_L_EXPIRY.dt.days

 

raw_df.loc[raw_df['DAYS_FROM_L_EXPIRY'] >= 0, 'EXPIRED_LICENSE'] = 0

raw_df.loc[raw_df['DAYS_FROM_L_EXPIRY'] < 0, 'EXPIRED_LICENSE'] = 1

In [10]:
# Days living at current address (date of loss - insurance_driver.date_at_current_address)

 

raw_df["DATE_AT_CURRENT_ADDRESS"] = pd.to_datetime(raw_df["DATE_AT_CURRENT_ADDRESS"])

raw_df["DAYS_AT_ADDRESS"] = raw_df["LOSS_EVENT_TIME"] - raw_df["DATE_AT_CURRENT_ADDRESS"]

raw_df["DAYS_AT_ADDRESS"] = abs(raw_df.DAYS_AT_ADDRESS.dt.days)

 

raw_df.loc[raw_df['DAYS_AT_ADDRESS'] >= 15, 'SUSPICIOUS_LIVING'] = 1

raw_df.loc[raw_df['DAYS_AT_ADDRESS'] < 15, 'SUSPICIOUS_LIVING'] = 0

In [11]:
raw_df["SUSPICIOUS_LIVING"].value_counts()

1.0    973
0.0      2
Name: SUSPICIOUS_LIVING, dtype: int64

In [12]:
#7500/year

raw_df["START_DATE"] = pd.to_datetime(raw_df["START_DATE"])

#find number of days between policy creation and accident

raw_df["LENGTH_OF_POLICY"]=(raw_df["LOSS_EVENT_TIME"] - raw_df["START_DATE"]).dt.days

 

#convert to years

raw_df["LENGTH_OF_POLICY"]=raw_df["LENGTH_OF_POLICY"]/365

 

#divide Odometer at loss by years

raw_df["MILES/YEAR"] = raw_df["ODOMETER_AT_LOSS"]/raw_df["LENGTH_OF_POLICY"]

raw_df["MILES/YEAR"].value_counts()

263963.479358    1
118223.849282    1
101693.055556    1
128489.623245    1
337031.306557    1
                ..
293520.237762    1
149085.527500    1
689559.489796    1
291911.041860    1
119873.677122    1
Name: MILES/YEAR, Length: 975, dtype: int64

In [13]:
# Conflict on whether a policyholder with a low mileage discount experienced a loss with high mileage at the point of loss

raw_df.loc[raw_df["MILES/YEAR"] <7500, 'LOW_MILEAGE_AT_LOSS'] = 1

raw_df.loc[raw_df["MILES/YEAR"] >=7500, 'LOW_MILEAGE_AT_LOSS'] = 0

In [14]:
raw_df.loc[raw_df["LOW_MILEAGE_USE"]==raw_df["LOW_MILEAGE_AT_LOSS"], 'SUSPICIOUS_MILEAGE'] = 0

raw_df.loc[raw_df["LOW_MILEAGE_USE"]!=raw_df["LOW_MILEAGE_AT_LOSS"], 'SUSPICIOUS_MILEAGE'] = 1

In [15]:
raw_df.loc[raw_df["CLAIM_AMOUNT"] <3000, 'EXCESSIVE_CLAIM_AMOUNT'] = 0

raw_df.loc[raw_df["CLAIM_AMOUNT"] >=3000, 'EXCESSIVE_CLAIM_AMOUNT'] = 1

In [16]:
# dataframes for certain features

features = ['FLAG_FOR_FRAUD_INV', 

 'SUSPICIOUS_MILEAGE',

 'EXPIRED_LICENSE',

 'SUSPICIOUS_CLAIM_TIME',

 'SUSPICIOUS_LIVING',

 'EXCESSIVE_CLAIM_AMOUNT']

In [17]:
df_model = raw_df[features]

In [18]:
#ensure all relevant features are integers

df_model["SUSPICIOUS_LIVING"] = df_model["SUSPICIOUS_LIVING"].astype(int)

df_model["EXPIRED_LICENSE"] = df_model["EXPIRED_LICENSE"].astype(int)

df_model["SUSPICIOUS_CLAIM_TIME"] = df_model["SUSPICIOUS_CLAIM_TIME"].astype(int)

df_model["SUSPICIOUS_MILEAGE"] = df_model["SUSPICIOUS_MILEAGE"].astype(int)

df_model["EXCESSIVE_CLAIM_AMOUNT"] = df_model["EXCESSIVE_CLAIM_AMOUNT"].astype(int)

/tmp/wsuser/ipykernel_573/1585913532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model["SUSPICIOUS_LIVING"] = df_model["SUSPICIOUS_LIVING"].astype(int)
/tmp/wsuser/ipykernel_573/1585913532.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model["EXPIRED_LICENSE"] = df_model["EXPIRED_LICENSE"].astype(int)
/tmp/wsuser/ipykernel_573/1585913532.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [19]:
raw_df.groupby("FLAG_FOR_FRAUD_INV", as_index=False).mean()

,FLAG_FOR_FRAUD_INV,INCIDENT_CAUSE,DESCRIPTION,CLAIM_STATUS,ODOMETER_AT_LOSS,POLICE_REPORT,CLAIMS_AT_LOSS_DATE,LOSS_LOCATION_LAT,LOSS_LOCATION_LONG,CLAIM_AMOUNT,...,SUSPICIOUS_CLAIM_TIME,DAYS_FROM_L_EXPIRY,EXPIRED_LICENSE,DAYS_AT_ADDRESS,SUSPICIOUS_LIVING,LENGTH_OF_POLICY,MILES/YEAR,LOW_MILEAGE_AT_LOSS,SUSPICIOUS_MILEAGE,EXCESSIVE_CLAIM_AMOUNT
0,0,2.926186,NaN,2.977153,180783.846046,0.281195,1.492091,41.857895,-87.675014,2521.147627,...,0.933216,894.546573,0.042179,3487.956063,0.996485,1.070159,205170.356701,0.0,0.094903,0.086116
1,1,3.041872,NaN,1.103448,175504.046305,0.637931,1.758621,41.860133,-87.679617,26429.480296,...,0.945813,917.960591,0.036946,3252.413793,1.000000,0.960490,230687.578997,0.0,0.088670,0.933498


In [20]:
#split data into x and y variables

xVar = df_model[["EXPIRED_LICENSE","SUSPICIOUS_CLAIM_TIME","SUSPICIOUS_LIVING","SUSPICIOUS_MILEAGE","EXCESSIVE_CLAIM_AMOUNT"]]

yVar = df_model["FLAG_FOR_FRAUD_INV"]

In [21]:
xVar.head()

,EXPIRED_LICENSE,SUSPICIOUS_CLAIM_TIME,SUSPICIOUS_LIVING,SUSPICIOUS_MILEAGE,EXCESSIVE_CLAIM_AMOUNT
0,0,1,1,0,1
1,1,1,1,0,0
2,0,1,1,1,1
3,0,1,1,0,1
4,0,1,1,0,0


In [22]:
#split into a test/train set

X_train, X_test, y_train, y_test = train_test_split(xVar, yVar, test_size=0.2)

print (X_train.shape, y_train.shape)

print (X_test.shape, y_test.shape)

(780, 5) (780,)
(195, 5) (195,)


In [23]:
#train model

clf = RandomForestClassifier(n_jobs=2, random_state=0)

 

clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=2, random_state=0)

In [24]:
#create confusion matrix to gut check model

preds = clf.predict(X_test)

pd.crosstab(y_test, preds, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,0,1
Actual Result,,
0,109,9
1,5,72
